In [51]:
import requests
import pandas as pd
import yaml
import os

from secrets_config import api_key


In [52]:
with open("config.yaml") as stream:
    config = yaml.safe_load(stream)

target_cities= config['extract']['target_cities']
salesResults_url = config['extract']['api_urls']['salesResults']
salesResults_listings_url = config['extract']['api_urls']['salesResults_listings']

In [60]:
pd.DataFrame(target_cities)

,0
0,Sydney
1,Melbourne


In [76]:
pd.concat([pd.DataFrame([['Sydney', url]], columns=['City', 'URL']), pd.json_normalize(data=response.json())], axis=1)

,City,URL,auctionedDate,lastModifiedDateTime,adjClearanceRate,median,numberAuctioned,numberListedForAuction,numberSold,numberUnreported,numberWithdrawn,totalSales
0,Sydney,https://api.domain.com.au/v1/salesResults/Sydney,2022-09-24,2022-09-28T04:01:34.631Z,0.6,1420000,585,725,351,11,130,357488588


In [68]:
params = {
    "api_key": api_key
}

##for city in target_cities:
    
response_data = []
url = salesResults_url + 'Sydney'

response = requests.get(url, params=params)

if response.json() is not None: 
    response_data.extend(response.json())
    print(f"Status: {response.status_code} for {url}")

df = pd.json_normalize(data=response.json())


Status: 200 for https://api.domain.com.au/v1/salesResults/Sydney


In [69]:
df


,auctionedDate,lastModifiedDateTime,adjClearanceRate,median,numberAuctioned,numberListedForAuction,numberSold,numberUnreported,numberWithdrawn,totalSales
0,2022-09-24,2022-09-28T04:01:34.631Z,0.6,1420000,585,725,351,11,130,357488588


In [28]:
#API_KEY = os.environ.get(f"{api_key}")

base_url = 'https://api.domain.com.au/v1/salesResults/Sydney/listings'

response_data = []

params = {
    "api_key": api_key
}
response = requests.get(base_url, params=params)
if response.json() is not None: 
    response_data.extend(response.json())
    print(response.status_code)

200


In [29]:
df_listings = pd.json_normalize(data=response_data)
df_listings.head()

,agencyId,id,propertyDetailsUrl,agencyName,agencyProfilePageUrl,agent,bathrooms,bedrooms,carspaces,postcode,...,result,state,streetName,streetNumber,streetType,suburb,geoLocation.latitude,geoLocation.longitude,price,unitNumber
0,33021,2018031516,https://www.domain.com.au/12-lanceley-place-ab...,Blaze Real Estate,https://www.domain.com.au/real-estate-agencies...,Blaze Real Estate - Wetherill Park,3,5,1,2176,...,AUSD,Nsw,Lanceley,12,Pl,Abbotsbury,-33.867078,150.867887,NaN,NaN
1,3812,2018038475,https://www.domain.com.au/5-glen-ormond-avenue...,Time Realty,https://www.domain.com.au/real-estate-agencies...,Time Realty,1,2,1,2046,...,AUSD,Nsw,Glen Ormond,5,Av,Abbotsford,-33.853453,151.128919,2540000.0,NaN
2,31998,2018008641,https://www.domain.com.au/10-tyler-crescent-ab...,DibChidiac,https://www.domain.com.au/real-estate-agencies...,DIB CHIDIAC,2,4,2,2046,...,AUSP,Nsw,Tyler,10,Cr,Abbotsford,-33.853034,151.135108,NaN,NaN
3,8692,2018047582,https://www.domain.com.au/7-kew-way-airds-nsw-...,Ray White Macarthur Group,https://www.domain.com.au/real-estate-agencies...,RW Macarthur Group,1,3,2,2560,...,AUSD,Nsw,Kew,7,Wy,Airds,-34.092862,150.821427,600000.0,NaN
4,8692,2018047583,https://www.domain.com.au/3-ponds-way-airds-ns...,Ray White Macarthur Group,https://www.domain.com.au/real-estate-agencies...,RW Macarthur Group,1,3,1,2560,...,AUSD,Nsw,Ponds,3,Wy,Airds,-34.093184,150.822937,650000.0,NaN


In [18]:
df_listings.columns

Index(['agencyId', 'id', 'propertyDetailsUrl', 'agencyName',
       'agencyProfilePageUrl', 'agent', 'bathrooms', 'bedrooms', 'carspaces',
       'postcode', 'propertyType', 'result', 'state', 'streetName',
       'streetNumber', 'streetType', 'suburb', 'geoLocation.latitude',
       'geoLocation.longitude', 'price', 'unitNumber'],
      dtype='object')

In [19]:
df_listings.dtypes

agencyId                   int64
id                         int64
propertyDetailsUrl        object
agencyName                object
agencyProfilePageUrl      object
agent                     object
bathrooms                  int64
bedrooms                   int64
carspaces                  int64
postcode                  object
propertyType              object
result                    object
state                     object
streetName                object
streetNumber              object
streetType                object
suburb                    object
geoLocation.latitude     float64
geoLocation.longitude    float64
price                    float64
unitNumber                object
dtype: object